In [214]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process

/Users/yagopajarino/Documents/uba/limpieza-datos/.venv/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [184]:
df = pd.read_excel("descargas/GSAF5.xls")

In [185]:
from xlrd import open_workbook
wb = open_workbook('descargas/GSAF5.xls', formatting_info=True)
sheet = wb.sheet_by_index(0)
bgcol=[]
for row in range(sheet.nrows):
    column = 1
    cell = sheet.cell(row, column)  
    fmt = wb.xf_list[cell.xf_index]
    bgcol.append(fmt.background.pattern_colour_index)


In [186]:
wb.colour_map[64]

In [187]:
set(bgcol)

{8, 18, 26, 42, 43, 44, 47, 53, 64}

In [188]:
# Esto hecho a mano
paleta = {
    8: "Orange",
    18: "Blue",
    26: "Tan",
    42: "Green",
    43: "Tan",
    44: "Blue",
    47: "Tan",
    53: "Orange",
    64: "White"
}

In [189]:
cell_colors = [paleta[x] for x in bgcol]

In [190]:
df["cell_color"] = pd.Series(cell_colors)

In [191]:
map_tipo_incidente = {
    "Tan": "Unprovoked Incidents",
    "Orange": "Provoked Incidents",
    "Green": "Attacks on Boats",
    "Yellow": "Air / Sea Disasters",
    "Blue": "Questionable Incidents",
    "White": "Other"
}

In [192]:
df["tipo_incidente"] = df["cell_color"].map(map_tipo_incidente)

In [193]:
df["tipo_incidente"].value_counts()

tipo_incidente
Unprovoked Incidents      5386
Provoked Incidents         638
Questionable Incidents     586
Attacks on Boats           363
Name: count, dtype: int64

In [194]:
df["Type"].value_counts(dropna=False)

Type
Unprovoked             5135
Provoked                636
Invalid                 552
Watercraft              355
Sea Disaster            242
Questionable             22
NaN                      18
Boat                      7
 Provoked                 2
Unconfirmed               1
?                         1
Unverified                1
Under investigation       1
Name: count, dtype: int64

In [195]:
df.columns

Index(['Date', 'Year', 'Type', 'Country', 'State', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Unnamed: 11', 'Time', 'Species ',
       'Source', 'pdf', 'href formula', 'href', 'Case Number', 'Case Number.1',
       'original order', 'Unnamed: 21', 'Unnamed: 22', 'cell_color',
       'tipo_incidente'],
      dtype='object')

In [196]:
df["Country"].value_counts()

Country
USA                   2551
AUSTRALIA             1481
SOUTH AFRICA           597
NEW ZEALAND            144
BAHAMAS                136
                      ... 
BAHREIN                  1
KOREA                    1
RED SEA?                 1
ASIA?                    1
CEYLON (SRI LANKA)       1
Name: count, Length: 235, dtype: int64

In [197]:
# trim country column
df["Country"] = df["Country"].str.strip()

In [198]:
# uppercase all
df["Country"] = df["Country"].str.upper()

In [200]:
# remove punctuiation
df['Country'] = df['Country'].str.replace(r'[^\w\s&\/]', '', regex=True)

In [201]:
sorted(df["Country"].dropna().unique())

['ADMIRALTY ISLANDS',
 'AFRICA',
 'ALGERIA',
 'AMERICAN SAMOA',
 'ANDAMAN / NICOBAR ISLANDAS',
 'ANDAMAN ISLANDS',
 'ANGOLA',
 'ANTIGUA',
 'ARGENTINA',
 'ARUBA',
 'ASIA',
 'ATLANTIC OCEAN',
 'AUSTRALIA',
 'AZORES',
 'BAHAMAS',
 'BAHREIN',
 'BANGLADESH',
 'BARBADOS',
 'BAY OF BENGAL',
 'BELIZE',
 'BERMUDA',
 'BETWEEN PORTUGAL & INDIA',
 'BRAZIL',
 'BRITISH ISLES',
 'BRITISH NEW GUINEA',
 'BRITISH OVERSEAS TERRITORY',
 'BRITISH VIRGIN ISLANDS',
 'BRITISH WEST INDIES',
 'BURMA',
 'CANADA',
 'CAPE VERDE',
 'CARIBBEAN SEA',
 'CAYMAN ISLANDS',
 'CENTRAL PACIFIC',
 'CEYLON',
 'CEYLON SRI LANKA',
 'CHILE',
 'CHINA',
 'COAST OF AFRICA',
 'COLOMBIA',
 'COLUMBIA',
 'COMOROS',
 'COOK ISLANDS',
 'CORAL SEA',
 'COSTA RICA',
 'CRETE',
 'CROATIA',
 'CUBA',
 'CURACAO',
 'CYPRUS',
 'DIEGO GARCIA',
 'DJIBOUTI',
 'DOMINICAN REPUBLIC',
 'ECUADOR',
 'EGYPT',
 'EGYPT / ISRAEL',
 'EL SALVADOR',
 'ENGLAND',
 'EQUATORIAL GUINEA / CAMEROON',
 'FALKLAND ISLANDS',
 'FEDERATED STATES OF MICRONESIA',
 'FIJI',
 'FRAN

In [204]:
# leo datos de countries https://github.com/google/dspl/blob/master/samples/google/canonical/countries.csv
countries = pd.read_csv("descargas/countries.csv")

In [213]:
# me quedo con nombre de paises que voy a usar para Levenshtein distance
standardized_countries = set(countries["name"].str.upper().str.strip())

In [217]:
df['Country'].isna().sum()

np.int64(50)

In [219]:
df.columns

Index(['Date', 'Year', 'Type', 'Country', 'State', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Unnamed: 11', 'Time', 'Species ',
       'Source', 'pdf', 'href formula', 'href', 'Case Number', 'Case Number.1',
       'original order', 'Unnamed: 21', 'Unnamed: 22', 'cell_color',
       'tipo_incidente'],
      dtype='object')

In [218]:
df[df["Country"].isna()]

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22,cell_color,tipo_incidente
702,Reported 13-Nov-2017,2017.0,Unprovoked,NaN,NaN,NaN,Surfing,Timur Yunusov,M,24,...,2017.11.13.R-Timur.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.11.13.R,2017.11.13.R,6270.0,NaN,NaN,Tan,Unprovoked Incidents
1168,Aug-2014,2014.0,Invalid,NaN,NaN,NaN,Sea disaster,Cuban refugees,M,NaN,...,2014.08.00-Cuban-refugees.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.08.00,2014.08.00,5803.0,NaN,NaN,Tan,Unprovoked Incidents
3604,Ca. 1983,1983.0,Unprovoked,NaN,English Channel,NaN,Swimming,Padma Shri Taranath Narayan Shenoy,M,NaN,...,1983.00.00.d-Shenoy.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1983.00.00.d,1983.00.00.d,3367.0,NaN,NaN,Tan,Unprovoked Incidents
4034,Nov-1970,1970.0,Unprovoked,NaN,NaN,NaN,NaN,Heinz Plotsky,M,NaN,...,1970.11.00-NV-Plotsky.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1970.11.00,1970.11.00,2937.0,NaN,NaN,Tan,Unprovoked Incidents
4043,02-Aug-1970,1970.0,Invalid,NaN,Caribbean Sea,Between St. Kitts & Nevis,Sea Disaster Sinking of ferryboat Christina,NaN,NaN,NaN,...,1970.08.02-Christina-ferryboat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1970.08.02,1970.08.02,2928.0,NaN,NaN,Tan,Unprovoked Incidents
4044,5-Jul-1970,1970.0,Unprovoked,NaN,NaN,NaN,NaN,male,M,NaN,...,1970.07.05-NV-male.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1970.07.05,1970.07.05,2927.0,NaN,NaN,Blue,Questionable Incidents
4051,Apr-1970,1970.0,Provoked,NaN,NaN,NaN,Freediving,Lionel Jarvis,M,NaN,...,1970.04.00.b-NV-Jarvis.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1970.04.00.b,1970.04.00.b,2920.0,NaN,NaN,Tan,Unprovoked Incidents
4055,5-Feb-1970,1970.0,Unprovoked,NaN,NaN,NaN,Wading,Sally Anne Irvine,F,8,...,1970.02.05-NV-Irvine.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1970.02.05,1970.02.05,2916.0,NaN,NaN,Tan,Unprovoked Incidents
4081,Aug-1969,1969.0,Unprovoked,NaN,NaN,NaN,NaN,Rodney Hughes,M,25,...,1969.08.00-NV-Hughes.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1969.08.00,1969.08.00,2890.0,NaN,NaN,Tan,Unprovoked Incidents
4261,21-Oct-1965,1965.0,Unprovoked,NaN,NaN,Florida Strait,The boat Caribou II sank,Mario Castellanos,M,39,...,1965.10.21-Castellanos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1965.10.21,1965.10.21,2710.0,NaN,NaN,Tan,Unprovoked Incidents


In [221]:
# elimino filas sin pais
df.dropna(subset=["Country"], inplace=True)

## Levenshtein distance

Uso una base de datos de nombres de países para calcular la distancia Levenshtein del país ingresado

In [222]:
# leo datos de countries https://github.com/google/dspl/blob/master/samples/google/canonical/countries.csv
countries = pd.read_csv("descargas/countries.csv")

In [241]:
countries["name"].sort_values()

2                                       Afghanistan
5                                           Albania
58                                          Algeria
11                                   American Samoa
0                                           Andorra
8                                            Angola
4                                          Anguilla
9                                        Antarctica
3                               Antigua and Barbuda
10                                        Argentina
6                                           Armenia
14                                            Aruba
13                                        Australia
12                                          Austria
15                                       Azerbaijan
29                                          Bahamas
22                                          Bahrain
18                                       Bangladesh
17                                         Barbados
33          

In [223]:
# me quedo con nombre de paises que voy a usar para Levenshtein distance
standardized_countries = set(countries["name"].str.upper().str.strip())

In [247]:
df["is_valid_country"] = df["Country"].isin(standardized_countries)

In [251]:
df["is_valid_country"].sum() / len(df)

np.float64(0.586595406615629)

In [255]:
df[~df["is_valid_country"]]["Country"].unique()

array(['USA', 'MALDIVE ISLANDS', 'TURKS AND CAICOS', 'TOBAGO', 'TRINIDAD',
       'CORAL SEA', 'ENGLAND', 'COLUMBIA', 'BRITISH OVERSEAS TERRITORY',
       'ST KITTS / NEVIS', 'ST MARTIN', 'REUNION ISLAND',
       'ST HELENA BRITISH OVERSEAS TERRITORY', 'REUNION',
       'ATLANTIC OCEAN', 'TRINIDAD & TOBAGO', 'DIEGO GARCIA', 'SCOTLAND',
       'ANTIGUA', 'TURKS & CAICOS', 'UNITED ARAB EMIRATES UAE', 'AZORES',
       'NEVIS', 'GULF OF ADEN', 'ST MAARTIN', 'GRAND CAYMAN',
       'CARIBBEAN SEA', 'OKINAWA', 'EGYPT / ISRAEL',
       'NORTHERN ARABIAN SEA', 'NORTH ATLANTIC OCEAN', 'SOUTH CHINA SEA',
       'WESTERN SAMOA', 'PACIFIC OCEAN', 'BRITISH ISLES', 'NEW BRITAIN',
       'JOHNSTON ISLAND', 'SOUTH PACIFIC OCEAN', 'NEW GUINEA', 'RED SEA',
       'NORTH PACIFIC OCEAN', 'FEDERATED STATES OF MICRONESIA',
       'MID ATLANTIC OCEAN', 'ADMIRALTY ISLANDS', 'BRITISH WEST INDIES',
       'SOUTH ATLANTIC OCEAN', 'PERSIAN GULF', 'RED SEA / INDIAN OCEAN',
       'NORTH SEA', 'ANDAMAN / NICOBAR ISL

In [254]:
threshold = 75

def map_country_name(value, choices):
    if value[1]:
        return value[0]
    best_match, score = process.extractOne(value[0], choices, scorer=fuzz.token_sort_ratio)
    return best_match if score >= threshold else value[0]  # Adjust the threshold as needed

# Apply the function to the "country" column
df['country_standardized'] = df[['Country', "is_valid_country"]].apply(lambda x: map_country_name(x, standardized_countries), axis=1)

/var/folders/gf/9p2mg5dd6pl1j3xx9354sj7r0000gp/T/ipykernel_1364/2370500645.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if value[1]:
/var/folders/gf/9p2mg5dd6pl1j3xx9354sj7r0000gp/T/ipykernel_1364/2370500645.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return value[0]
/var/folders/gf/9p2mg5dd6pl1j3xx9354sj7r0000gp/T/ipykernel_1364/2370500645.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  best_match, score = proces

In [256]:
df["no_coincide"] = df["Country"] != df["country_standardized"]

In [257]:
# pandas display all rows
pd.set_option('display.max_rows', None)

In [258]:
df[df["no_coincide"]][["Country", "country_standardized"]].drop_duplicates()

,Country,country_standardized
19,TURKS AND CAICOS,TURKS AND CAICOS ISLANDS
22,TOBAGO,TOGO
178,ENGLAND,GREENLAND
222,COLUMBIA,COLOMBIA
265,BRITISH OVERSEAS TERRITORY,BRITISH INDIAN OCEAN TERRITORY
349,ST KITTS / NEVIS,SAINT KITTS AND NEVIS
770,REUNION,RÉUNION
1226,TRINIDAD & TOBAGO,TRINIDAD AND TOBAGO
1569,UNITED ARAB EMIRATES UAE,UNITED ARAB EMIRATES
2184,GRAND CAYMAN,CAYMAN ISLANDS
